In [1]:
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [2]:
import glob

In [3]:
files = glob.glob('*.txt')
reviews = ""
for i in range(len(files)):
    file = open(files[i],'r')
    data = file.read()
    table = str.maketrans('','',string.punctuation)
    data = data.lower().translate(table)
    reviews += data
    file.close()

In [4]:
with open('dataset.txt', 'w') as file:
    file.write(reviews)

In [5]:
# imdb = pd.read_csv('imdb_labelled.txt', sep='\t', header=None)
# TSV - Tab Seperated Values
df = pd.read_csv('dataset.txt', sep='\t', header=None, encoding='latin1')

In [6]:
# imdb.head()
df.head()

,0,1
0,so there is no way for me to plug it in here i...,0
1,good case excellent value,1
2,great for the jawbone,1
3,tied to charger for conversations lasting more...,0
4,the mic is great,1


In [7]:
# amazon = pd.read_csv('amazon_cells_labelled.txt', sep='\t', header=None)
# yelp = pd.read_csv('yelp_labelled.txt', sep='\t', header=None)

In [8]:
# df = pd.DataFrame()

In [9]:
# df = df.append(imdb)
# df = df.append(amazon)
# df = df.append(yelp)

In [10]:
df.shape

(3000, 2)

In [11]:
df.columns = ['Review','Sentiment']

In [12]:
df.head()

,Review,Sentiment
0,so there is no way for me to plug it in here i...,0
1,good case excellent value,1
2,great for the jawbone,1
3,tied to charger for conversations lasting more...,0
4,the mic is great,1


In [13]:
def textProcessing(df):
    table = str.maketrans('"',' ',string.punctuation)
    for i in range(len(df)):
        df['Review'].iloc[i] = df['Review'].iloc[i].lower().translate(table)
    
    tokens = []
    for i in range(len(df)):
        tokens.append(word_tokenize(df['Review'].iloc[i]))
    
    eng_stopwords = stopwords.words('english')
    wordsList = []
    for i in range(len(tokens)):
        tokensList = []
        for word in tokens[i]:
            if word not in eng_stopwords:
                tokensList.append(word)
        wordsList.append(tokensList)
        
    wnet = WordNetLemmatizer()
    for i in range(len(wordsList)):
        for j in range(len(wordsList[i])):
            wordsList[i][j] = wnet.lemmatize(wordsList[i][j], pos='v')
    for i in range(len(wordsList)):
        wordsList[i] = ' '.join(wordsList[i])
    
    return wordsList

In [14]:
wordsList = textProcessing(df)

C:\Users\asus\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
len(wordsList)

3000

In [16]:
wordsList[1]

'good case excellent value'

In [17]:
tfidf = TfidfVectorizer()

In [18]:
vectors = tfidf.fit_transform(wordsList).toarray()

In [19]:
vectors.shape

(3000, 4536)

In [20]:
y = df['Sentiment']

In [22]:
x_train, x_test, y_train, y_test = train_test_split(vectors,y,test_size=0.25)

In [23]:
gb = GaussianNB()
gb.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [24]:
y_pred = gb.predict(x_test)

In [25]:
accuracy_score(y_test, y_pred)

0.648

In [26]:
mb = MultinomialNB()
mb.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [27]:
y_pred = mb.predict(x_test)

In [28]:
accuracy_score(y_test, y_pred)

0.8133333333333334

In [29]:
test_review = "that movie was really very good"

In [32]:
test_df = pd.DataFrame({'Review':[test_review]})
test_wordList = textProcessing(test_df)

In [33]:
test_array = tfidf.transform(test_wordList)

In [35]:
test_array = test_array.toarray()

In [36]:
mb.predict(test_array)

array([1], dtype=int64)